# Explotation: LE RETOUR
<img width=300px src="https://pbs.twimg.com/profile_images/496812657341890561/m9vimoug.jpeg">

In [1]:
import torch
from torch.utils.data import DataLoader
from src.final_model import WAE, AudioDataset

from src.util import audio_utilities as au

import librosa as li

from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

from IPython.display import Audio

import numpy as np

model = torch.load("pretrained/model_20000_epoch.pt")
model.eval()
motu  = DataLoader(AudioDataset("notebooks/motu/motu_dataset/*.wav", process=True, slice_size=128), batch_size=100, shuffle=True, drop_last=True)

/home/antoine/lap/lib/python3.6/site-packages/torch/serialization.py:434: SourceChangeWarning: source code of class 'src.final_model.WAE' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


Preprocessing stuff... 
===========Done!


In [ ]:
def show_spec(X):
    """Affiche un spectrogramme"""
    plt.figure(figsize=(15,10))
    plt.imshow(X, origin="lower", aspect="auto")
    plt.show()
    
def GL_1pass(X, partial=False):
    """Une passe de Griffin Lim. L'argument partial définit si l'intégralité de la phase est recherchée
    ou si juste la motié droite du spectrogramme est reconstruite"""
    N = X.shape[0]
    
    x = au.istft_for_reconstruction(X, 2048, 256)
    X_ = au.stft_for_reconstruction(x, 2048, 256)
    
    if partial:
        angle = np.angle(X_)
        Y = abs(X).astype(complex)
        Y[N//2:] *= np.exp(1j*angle[N//2:])
        Y[:N//2] = X[:N//2]
    else:
        angle = np.angle(X_)
        Y = abs(X).astype(complex)
        Y *= np.exp(1j*angle)
        
    return Y

def GL(X, iteration, partial=False):
    """Fais pleins de fois GL_1pass"""
    X = X.astype(complex)
    for i in range(iteration):
        X = GL_1pass(X, partial=partial)
    return X

def rec(X):
    return au.istft_for_reconstruction(X, 2048, 256)
    
    
    
mel = li.filters.mel(22050, 2048, 500)
#X = mel.T.dot(X).T.astype(complex)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

for idx, (minibatch, f) in enumerate(motu):
    print(f)
    minibatch = minibatch.to(device)
    z = model.encode(minibatch)

z = z.detach().cpu().numpy()

del(minibatch)
torch.cuda.empty_cache()

In [ ]:
z_embedded = TSNE().fit_transform(z)
print("Embedded latent space of dimension %d into %d dimensions" % (z.shape[1], z_embedded.shape[1]))

In [ ]:
plt.figure(figsize=(10,10))
plt.plot(z_embedded[:,0], z_embedded[:,1], '.')
plt.show()

In [ ]:
gp1_x = z_embedded[:,0] < -4.5
gp1_y = z_embedded[:,1] > 6

gp1   = z[gp1_x*gp1_y,:]

print(gp1.shape)

In [ ]:
model.n_trames = 128
out = model.decode(torch.from_numpy(gp1).to(device)).detach().cpu().numpy()

In [ ]:
liste_audio = []

for i in range(out.shape[0]):
    print(i, end="", flush=True)
    X = mel.T.dot(out[i,:,:]).T
    liste_audio.append(rec(GL(X,300)))

In [ ]:
y = np.concatenate(liste_audio)
Audio(y, rate=22050)

In [ ]:
au.save_audio_to_file(y/max(abs(y)), 22050, outfile="pate1.wav")

In [ ]:
print(boolean)

In [ ]:
x,fs = li.load("notebooks/motu/motu_dataset/motu_sample_008.wav")

In [ ]:
xx = np.correlate(x,x, mode="full")[len(x):]
fmin = 100
fmax = 2000

tmin = fs//fmax
tmax = fs//fmin

f = 1/(np.argmax(xx[tmin:tmax])+tmin)

In [ ]:
print(f)

In [ ]:
x_ = np.fft.fft(x)
f_ = np.linspace(0,fs,len(x_))

plt.plot(f_,np.log(abs(x_)))
plt.axis([100,1000,None,None])

In [2]:
GC = AudioDataset("notebooks/motu/motu_dataset/*.wav", process=False, slice_size=128)
GC[4]

(tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 9.5320e-03, 9.1242e-03,
          6.6580e-03],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.5626e-02, 1.4958e-02,
          1.0915e-02],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 6.3098e-03, 7.2415e-03,
          5.9576e-03],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 9.1664e-07, 9.6390e-07,
          8.6701e-07],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 1.1626e-06, 1.2333e-06,
          1.1112e-06],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 8.6457e-07, 9.9605e-07,
          1.0791e-06]]), 0.017241379310344827)